In [1]:
import requests
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup

### Problem Statement: 
Using championship player data from the past 40 years, can we build a "championship" roster? 

Audience: NBA team executive

Limitations to address: salary cap or no?

In [34]:
#regular stats per game:
url = 'https://www.basketball-reference.com/playoffs/'
res = requests.get(url)
res.status_code
soup = BeautifulSoup(res.content, 'lxml')
table = soup.find('table', {'id': 'champions_index'})
champions=table.find('tbody').find_all('td', {'data-stat': 'champion'})
champ_years=table.find('tbody').find_all('th', {'data-stat': 'year_id'})
players = []
for i in range(41):
    url_2 = 'https://www.basketball-reference.com' + table.find('tbody').find_all('td', {'data-stat': 'champion'})[i].find('a')['href']
    res_2 = requests.get(url_2)
    res_2.status_code
    soup_2 = BeautifulSoup(res_2.content, 'lxml')
    table_2 = soup_2.find('table', {'id': 'per_game'})
    for row in table_2.find('tbody').find_all('tr'):
        player = {}
        player['name'] = row.find('td').text
        player['team'] = champions[i].text
        player['year'] = champ_years[i].text
        player['mpg'] = row.find('td', {'data-stat': 'mp_per_g'}).text
        player['fg%'] = row.find('td', {'data-stat': 'fg_pct'}).text
        player['3pfg%'] = row.find('td', {'data-stat': 'fg3_pct'}).text
        player['rebounds'] = row.find('td', {'data-stat': 'trb_per_g'}).text
        player['assists'] = row.find('td', {'data-stat': 'ast_per_g'}).text
        player['steals'] = row.find('td', {'data-stat': 'stl_per_g'}).text
        player['blocks'] = row.find('td', {'data-stat': 'blk_per_g'}).text
        player['turnovers'] = row.find('td', {'data-stat': 'tov_per_g'}).text
        player['points_per_game'] = row.find('td', {'data-stat': 'pts_per_g'}).text
        players.append(player)
        
time.sleep(5)
players
        
        

[{'name': 'LeBron James',
  'team': 'Los Angeles Lakers',
  'year': '2020',
  'mpg': '34.6',
  'fg%': '.493',
  '3pfg%': '.348',
  'rebounds': '7.8',
  'assists': '10.2',
  'steals': '1.2',
  'blocks': '0.5',
  'turnovers': '3.9',
  'points_per_game': '25.3'},
 {'name': 'Anthony Davis',
  'team': 'Los Angeles Lakers',
  'year': '2020',
  'mpg': '34.4',
  'fg%': '.503',
  '3pfg%': '.330',
  'rebounds': '9.3',
  'assists': '3.2',
  'steals': '1.5',
  'blocks': '2.3',
  'turnovers': '2.5',
  'points_per_game': '26.1'},
 {'name': 'Kentavious Caldwell-Pope',
  'team': 'Los Angeles Lakers',
  'year': '2020',
  'mpg': '25.5',
  'fg%': '.467',
  '3pfg%': '.385',
  'rebounds': '2.1',
  'assists': '1.6',
  'steals': '0.8',
  'blocks': '0.2',
  'turnovers': '0.9',
  'points_per_game': '9.3'},
 {'name': 'Kyle Kuzma',
  'team': 'Los Angeles Lakers',
  'year': '2020',
  'mpg': '25.0',
  'fg%': '.436',
  '3pfg%': '.316',
  'rebounds': '4.5',
  'assists': '1.3',
  'steals': '0.5',
  'blocks': '0.4',
 

In [35]:
df=pd.DataFrame(players)
df

,name,team,year,mpg,fg%,3pfg%,rebounds,assists,steals,blocks,turnovers,points_per_game
0,LeBron James,Los Angeles Lakers,2020,34.6,.493,.348,7.8,10.2,1.2,0.5,3.9,25.3
1,Anthony Davis,Los Angeles Lakers,2020,34.4,.503,.330,9.3,3.2,1.5,2.3,2.5,26.1
2,Kentavious Caldwell-Pope,Los Angeles Lakers,2020,25.5,.467,.385,2.1,1.6,0.8,0.2,0.9,9.3
3,Kyle Kuzma,Los Angeles Lakers,2020,25.0,.436,.316,4.5,1.3,0.5,0.4,1.5,12.8
4,Danny Green,Los Angeles Lakers,2020,24.8,.416,.367,3.3,1.3,1.3,0.5,0.9,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
637,Don Ford,Los Angeles Lakers,1980,11.2,.508,.000,1.9,0.7,0.2,0.3,0.6,3.0
638,Marty Byrnes,Los Angeles Lakers,1980,6.1,.500,,0.8,0.4,0.2,0.0,0.7,2.0
639,Ollie Mack,Los Angeles Lakers,1980,5.7,.420,.000,0.8,0.7,0.1,0.0,0.3,1.9
640,Brad Holland,Los Angeles Lakers,1980,5.2,.423,.200,0.4,0.6,0.4,0.0,0.3,2.8


In [37]:
df.to_csv('per_game_scrape.csv')

In [65]:
#roster scrape:
url = 'https://www.basketball-reference.com/playoffs/'
res = requests.get(url)
res.status_code
soup = BeautifulSoup(res.content, 'lxml')
table = soup.find('table', {'id': 'champions_index'})
champions=table.find('tbody').find_all('td', {'data-stat': 'champion'})
champ_years=table.find('tbody').find_all('th', {'data-stat': 'year_id'})
rosters = []
for i in range(41):
    url_2 = 'https://www.basketball-reference.com' + table.find('tbody').find_all('td', {'data-stat': 'champion'})[i].find('a')['href']
    res_2 = requests.get(url_2)
    res_2.status_code
    soup_2 = BeautifulSoup(res_2.content, 'lxml')
    table_3 = soup_2.find('table', {'id': 'roster'})
    for row in table_3.find('tbody').find_all('tr'):
        player_roster = {}
        player_roster['name'] = row.find('td', {'data-stat': 'player'}).text
        player_roster['year'] = champ_years[i].text
        player_roster['position'] = row.find('td', {'data-stat': 'pos'}).text
        rosters.append(player_roster)

time.sleep(5)
rosters

[{'name': 'Kostas Antetokounmpo', 'year': '2020', 'position': 'PF'},
 {'name': 'Avery Bradley', 'year': '2020', 'position': 'SG'},
 {'name': 'Devontae Cacok', 'year': '2020', 'position': 'C'},
 {'name': 'Kentavious Caldwell-Pope', 'year': '2020', 'position': 'SG'},
 {'name': 'Alex Caruso', 'year': '2020', 'position': 'PG'},
 {'name': 'Quinn Cook', 'year': '2020', 'position': 'PG'},
 {'name': 'Troy Daniels', 'year': '2020', 'position': 'SG'},
 {'name': 'Anthony Davis', 'year': '2020', 'position': 'PF'},
 {'name': 'Jared Dudley', 'year': '2020', 'position': 'PF'},
 {'name': 'Danny Green', 'year': '2020', 'position': 'SG'},
 {'name': 'Talen Horton-Tucker', 'year': '2020', 'position': 'SG'},
 {'name': 'Dwight Howard', 'year': '2020', 'position': 'C'},
 {'name': 'LeBron James', 'year': '2020', 'position': 'PG'},
 {'name': 'Kyle Kuzma', 'year': '2020', 'position': 'PF'},
 {'name': 'JaVale McGee', 'year': '2020', 'position': 'C'},
 {'name': 'Markieff Morris', 'year': '2020', 'position': 'PF'}

In [66]:
roster_df = pd.DataFrame(rosters)
roster_df

,name,year,position
0,Kostas Antetokounmpo,2020,PF
1,Avery Bradley,2020,SG
2,Devontae Cacok,2020,C
3,Kentavious Caldwell-Pope,2020,SG
4,Alex Caruso,2020,PG
...,...,...,...
637,Mark Landsberger,1980,C
638,Butch Lee,1980,PG
639,Ollie Mack,1980,SG
640,Norm Nixon,1980,PG


In [67]:
roster_df.to_csv('roster.csv')

In [68]:
#advanced metrics scrape:
url = 'https://www.basketball-reference.com/playoffs/'
res = requests.get(url)
res.status_code
soup = BeautifulSoup(res.content, 'lxml')
table = soup.find('table', {'id': 'champions_index'})
champions=table.find('tbody').find_all('td', {'data-stat': 'champion'})
champ_years=table.find('tbody').find_all('th', {'data-stat': 'year_id'})
advanced = []
for i in range(41):
    url_2 = 'https://www.basketball-reference.com' + table.find('tbody').find_all('td', {'data-stat': 'champion'})[i].find('a')['href']
    res_2 = requests.get(url_2)
    res_2.status_code
    soup_2 = BeautifulSoup(res_2.content, 'lxml')
    table_4 = soup_2.find('table', {'id': 'advanced'})
    for row in table_4.find('tbody').find_all('tr'):
        player_advanced = {}
        player_advanced['name'] = row.find('td', {'data-stat': 'player'}).text
        player_advanced['year'] = champ_years[i].text
        player_advanced['player_efficiency'] = row.find('td', {'data-stat': 'per'}).text
        advanced.append(player_advanced)

time.sleep(5)
advanced

[{'name': 'LeBron James', 'year': '2020', 'player_efficiency': '25.5'},
 {'name': 'Anthony Davis', 'year': '2020', 'player_efficiency': '27.4'},
 {'name': 'Kentavious Caldwell-Pope',
  'year': '2020',
  'player_efficiency': '11.0'},
 {'name': 'Danny Green', 'year': '2020', 'player_efficiency': '10.8'},
 {'name': 'Kyle Kuzma', 'year': '2020', 'player_efficiency': '12.2'},
 {'name': 'Dwight Howard', 'year': '2020', 'player_efficiency': '18.7'},
 {'name': 'Avery Bradley', 'year': '2020', 'player_efficiency': '8.9'},
 {'name': 'Alex Caruso', 'year': '2020', 'player_efficiency': '10.8'},
 {'name': 'JaVale McGee', 'year': '2020', 'player_efficiency': '19.4'},
 {'name': 'Rajon Rondo', 'year': '2020', 'player_efficiency': '12.4'},
 {'name': 'Quinn Cook', 'year': '2020', 'player_efficiency': '10.0'},
 {'name': 'Troy Daniels', 'year': '2020', 'player_efficiency': '9.0'},
 {'name': 'Jared Dudley', 'year': '2020', 'player_efficiency': '7.8'},
 {'name': 'Markieff Morris', 'year': '2020', 'player_ef

In [69]:
df_advanced = pd.DataFrame(advanced)
df_advanced

,name,year,player_efficiency
0,LeBron James,2020,25.5
1,Anthony Davis,2020,27.4
2,Kentavious Caldwell-Pope,2020,11.0
3,Danny Green,2020,10.8
4,Kyle Kuzma,2020,12.2
...,...,...,...
637,Marty Byrnes,1980,6.3
638,Ollie Mack,1980,9.1
639,Ron Boone,1980,3.0
640,Kenny Carr,1980,5.2


In [43]:
df_advanced.to_csv('advanced.csv')

In [52]:
df.sort_values('name')

,name,team,year,mpg,fg%,3pfg%,rebounds,assists,steals,blocks,turnovers,points_per_game
539,A.C. Green,Los Angeles Lakers,1987,28.4,.538,.000,7.8,1.1,0.9,1.0,1.3,10.8
523,A.C. Green,Los Angeles Lakers,1988,32.1,.503,.000,8.7,1.1,1.1,0.5,1.5,11.4
340,A.C. Green,Los Angeles Lakers,2000,23.5,.447,.250,5.9,1.0,0.6,0.2,0.6,5.0
207,Adam Morrison,Los Angeles Lakers,2009,5.5,.333,.250,1.0,0.4,0.0,0.0,0.4,1.3
191,Adam Morrison,Los Angeles Lakers,2010,7.8,.376,.238,1.0,0.6,0.1,0.1,0.3,2.4
...,...,...,...,...,...,...,...,...,...,...,...,...
19,Zach Norvell,Los Angeles Lakers,2020,2.5,.000,,0.5,0.0,0.0,0.0,0.0,0.0
52,Zaza Pachulia,Golden State Warriors,2018,14.1,.564,.000,4.7,1.6,0.6,0.2,1.0,5.4
65,Zaza Pachulia,Golden State Warriors,2017,18.1,.534,.000,5.9,1.9,0.8,0.5,1.2,6.1
427,Žan Tabak,Houston Rockets,1995,4.9,.453,.000,1.5,0.1,0.1,0.2,0.5,2.0


In [70]:
#https://stackoverflow.com/questions/41815079/pandas-merge-join-two-data-frames-on-multiple-columns
new_df = pd.merge(df, df_advanced,  how='inner', on=['name','year'])

In [72]:
new_df[new_df['name']=='Randy Brown']

,name,team,year,mpg,fg%,3pfg%,rebounds,assists,steals,blocks,turnovers,points_per_game,player_efficiency
372,Randy Brown,Chicago Bulls,1998,16.2,.384,.000,1.3,2.1,1.0,0.2,0.9,4.1,10.1
391,Randy Brown,Chicago Bulls,1997,14.7,.420,.182,1.5,1.8,1.1,0.2,0.8,4.7,13.0
408,Randy Brown,Chicago Bulls,1996,9.9,.406,.091,1.0,1.1,0.8,0.2,0.5,2.7,11.2


In [73]:
final_df = pd.merge(new_df, roster_df,  how='inner', on=['name','year'])

In [74]:
final_df

,name,team,year,mpg,fg%,3pfg%,rebounds,assists,steals,blocks,turnovers,points_per_game,player_efficiency,position
0,LeBron James,Los Angeles Lakers,2020,34.6,.493,.348,7.8,10.2,1.2,0.5,3.9,25.3,25.5,PG
1,Anthony Davis,Los Angeles Lakers,2020,34.4,.503,.330,9.3,3.2,1.5,2.3,2.5,26.1,27.4,PF
2,Kentavious Caldwell-Pope,Los Angeles Lakers,2020,25.5,.467,.385,2.1,1.6,0.8,0.2,0.9,9.3,11.0,SG
3,Kyle Kuzma,Los Angeles Lakers,2020,25.0,.436,.316,4.5,1.3,0.5,0.4,1.5,12.8,12.2,PF
4,Danny Green,Los Angeles Lakers,2020,24.8,.416,.367,3.3,1.3,1.3,0.5,0.9,8.0,10.8,SG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,Don Ford,Los Angeles Lakers,1980,11.2,.508,.000,1.9,0.7,0.2,0.3,0.6,3.0,8.3,SF
638,Marty Byrnes,Los Angeles Lakers,1980,6.1,.500,,0.8,0.4,0.2,0.0,0.7,2.0,6.3,SF
639,Ollie Mack,Los Angeles Lakers,1980,5.7,.420,.000,0.8,0.7,0.1,0.0,0.3,1.9,9.1,SG
640,Brad Holland,Los Angeles Lakers,1980,5.2,.423,.200,0.4,0.6,0.4,0.0,0.3,2.8,14.3,SG


In [77]:
final_df.to_csv('merged.csv')